In [1]:
from typing import MutableMapping
from estnltk.taggers import Tagger
from estnltk import Layer, Text, Span, BaseSpan, ElementaryBaseSpan

from pipelines.step03_BERT_fine_tuning import TokenClassification
from pipelines.step03_BERT_fine_tuning.dataloaders import Tokens

from transformers import AutoTokenizer, BertForTokenClassification, logging, BertForPreTraining
from transformers import pipeline

from estnltk_patches.taggers.ner_tagger.DynamicTagger import DynamicTagger

import os


In [2]:
tokenizer_args = {
    "lowercase": False
}
tokenization_args = {
    "max_length": 128,
    "padding": "max_length",
    "truncation": True,
    'is_split_into_words': True
}

pretrained_model_path = "medbert_models/pretr_copy"
map_args = {
    "batched": True
}
# model = BertForPreTraining.from_pretrained(pretrained_model_path)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path, **tokenizer_args)

def generate_n_unk_tokens(number_of_tokens: int):
    list_of_tokens = ['[UNK' + str(k) + ']' for k in range(1, number_of_tokens+1)]
    return {'additional_special_tokens': list_of_tokens}

special_tokens_dict = generate_n_unk_tokens(100)
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
# model.resize_token_embeddings(len(tokenizer))

#
# model = BertForTokenClassification.from_pretrained(pretrained_model_path)
#
# TokenClassification.finetune_BERT(pretrained_model_path, dl, False, True, map_args, tokenizer_args,
#                                   tokenization_args, training_args)
#
# res = TokenClassification.evaluate(pretrained_model_path, dl, False, map_args, tokenizer_args,
#                                    tokenization_args, training_args)
#
# tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path, tokenizer_args)
# model = BertForTokenClassification.from_pretrained(finetuned_model_path)
# self.pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [3]:
tokenizer.save_pretrained('tokenizer_100_additional_UNKS')

('tokenizer_100_additional_UNKS/tokenizer_config.json',
 'tokenizer_100_additional_UNKS/special_tokens_map.json',
 'tokenizer_100_additional_UNKS/vocab.txt',
 'tokenizer_100_additional_UNKS/added_tokens.json',
 'tokenizer_100_additional_UNKS/tokenizer.json')

In [5]:
# Indeed, it does not tokenize [UNK1] and [UNK2] now
tokenizer.tokenize("hello my name is [UNK1] and [UNK2] and [UNK11010]")

['hell',
 '##o',
 'my',
 'n',
 '##ame',
 'i',
 '##s',
 '[UNK1]',
 'and',
 '[UNK2]',
 'and',
 '[',
 'u',
 '##n',
 '##k',
 '##110',
 '##10',
 ']']

In [6]:
# Loading up the tokenizer from the pretrained model to check if it still does not tokenize the specific UNK tokens
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path, **tokenizer_args)
tokenizer.tokenize("hello my name is [UNK1] and [UNK2] and [UNK11010]")


['hell',
 '##o',
 'my',
 'n',
 '##ame',
 'i',
 '##s',
 '[UNK1]',
 'and',
 '[UNK2]',
 'and',
 '[',
 'u',
 '##n',
 '##k',
 '##110',
 '##10',
 ']']

In [16]:
finetuned_model_path = "medbert_models/testing_additional_tokens_tokmodel"

training_data = "botrk2/7.1642407884/1647506105/stuckinthemiddlewithyou.tsv"

dl = Tokens.Tsv(training_data)

print(dl.read())

training_args = {
    "output_dir": finetuned_model_path,
    "overwrite_output_dir": True,
    "num_train_epochs": 1,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 8
}
TokenClassification.finetune_BERT(pretrained_model_path, dl, False, True, map_args, tokenizer_args,
                                              tokenization_args, training_args)

loading file medbert_models/pretrained_copy/vocab.txt
loading file medbert_models/pretrained_copy/tokenizer.json
loading file medbert_models/pretrained_copy/added_tokens.json
loading file medbert_models/pretrained_copy/special_tokens_map.json
loading file medbert_models/pretrained_copy/tokenizer_config.json


Dataset({
    features: ['X', 'y'],
    num_rows: 1
})


loading configuration file medbert_models/pretrained_copy/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 50000
}

loading weights file medbert_models/pretrained_copy/pytorch_model.bin


Some weights of the model checkpoint at medbert_models/pretrained_copy were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initial

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to medbert_models/testing_additional_tokens_tokmodel
Configuration saved in medbert_models/testing_additional_tokens_tokmodel/config.json
Model weights saved in medbert_models/testing_additional_tokens_tokmodel/pytorch_model.bin
tokenizer config file saved in medbert_models/testing_additional_tokens_tokmodel/tokenizer_config.json
Special tokens file saved in medbert_models/testing_additional_tokens_tokmodel/special_tokens_map.json


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

{'additional_special_tokens': ['[UNK1]',
  '[UNK2]',
  '[UNK3]',
  '[UNK4]',
  '[UNK5]']}